In [158]:
import cartoframes
import pandas as pd
pd.options.display.max_columns = 200

import requests as req
import json
import boto3
import io
import datetime
from dateutil import parser

import sys
import logging
import os

ACCESS_ID = os.environ.get('S3_ACCESS_ID')
SECRET_KEY = os.environ.get('S3_ACCESS_KEY')

s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_ID,
    aws_secret_access_key=SECRET_KEY
)
s3_resource = boto3.resource(
    's3',
    aws_access_key_id=ACCESS_ID,
    aws_secret_access_key=SECRET_KEY
)

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/wide_to_long/"

# Functions for reading and uploading data to/from S3
def read_from_S3(bucket, key, index_col=0):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=[index_col], encoding="utf8")
    return(df)

def write_to_S3(df, bucket, key):
    csv_buffer = io.StringIO()
    # Need to set encoding in Python2... default of 'ascii' fails
    df.to_csv(csv_buffer, encoding='utf-8')
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())


# Set up carto context
user = 'wri-rw' # 'rw-nrt'
key = {
    'wri-rw':'',
    'rw-nrt':''
}
cc = cartoframes.CartoContext(base_url='https://{}.carto.com/'.format(user),
                              api_key=key[user])

# Access data from API
# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[dset["id"]] = {
        "name":atts["name"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

# Carto datasets
provider = "cartodb"
carto_ids = (current_datasets_on_api["provider"]==provider)
carto_data = current_datasets_on_api.loc[carto_ids]

logging.info("Number of Carto datasets: " + str(carto_data.shape[0]))


# Long form table possibilities
long_forms = {"cit.022":"6b670396-c52c-430c-b5bb-20693da03b60","cit.025":"d38d0d5c-31b1-47f4-9d2e-d8fba4c7d083","cit.028":"35ce2b98-adbb-4873-b334-d7b1cc542de7","cit.029":"10337db6-8321-445e-a60b-28fc1e114f29","cli.007":"3d2ce960-abda-4c9c-bd29-1929e9ca24c9","cli.008":"3a46f6b4-0eec-49d4-bbfc-e2e8f64e6117","com.006":"2e31a1f3-576b-46b4-84f0-3f0cc399f887","com.007":"fe311144-8c0e-4440-b068-6efd057e0f6a","com.009":"c61c364b-1d68-4dd9-ae3d-76c2a0022280","com.010":"52c55378-0484-48c3-92fc-3ee94d21c716","com.015":"c18a38cd-94ff-48cd-818f-6ffb05992abb","com.019":"5e3a3a9f-7380-47c0-ad84-2c193861e106","com.028":"62c988a7-1e4d-418e-87bf-a743e24209e8","ene.012":"d446a52e-c4c1-4e74-ae30-3204620a0365","ene.021":"2c092793-aa3a-4520-959c-ad48165dcae4","ene.022":"d639909f-bcf3-4875-b8c3-35f030b68ed3","ene.028":"c665f519-eef9-4f67-a8bf-7e3e6dc8bfcd","foo.006":"2034a766-6e8a-416d-b8ab-9b7b3e3abb15","foo.015":"4338471d-881a-475f-8bd9-60c4d48b8e12","foo.019":"8bc79a36-d77e-4ee3-b9bc-c77146cfc503","foo.040":"91ff1359-6680-49bc-8002-20256e999993","foo.041":"ccfb322a-20aa-4132-b58b-0f76acec8f5a","foo.042":"7a551dd8-b59c-4f59-9d50-c92cb61c5799","foo.043":"95b013a3-389a-4367-83b7-c9d68c28c406","for.021":"05b7c688-09ba-4f33-90ea-185a1039df43","soc.004":"bea122ce-1e4b-465d-8b7b-fa11aadd20f7","cit.017":"0303127a-70b0-4164-9251-d8162615d058","soc.005":"a7067e9f-fe40-4338-85da-13a6071c76fe","soc.006":"a89c95c7-0b82-4162-b9d8-cc0205e9f7ec","soc.008":"00abb46f-34e2-4bf7-be30-1fb0b1de022f","soc.015":"e8f53f73-d77c-485a-a2a6-1c47ea4aead9","soc.020":"f8d3e79c-c3d0-4f9a-9b68-9c5ad1f025e4","soc.023":"d3a6b89f-cf5c-40cf-b2b3-ac1c8315c648","soc.025":"11278cb6-b298-49a1-bf71-f1e269f40758","soc.029":"7793f46c-a48a-466f-a8ce-ca1a87b7aeed","soc.035":"e7780d53-ad80-45bd-a271-79615ee97a37","soc.036":"8671f536-1979-4b6f-a147-70152fcb44ed","soc.039":"b37048be-9b23-4458-a047-888956c69aa1","soc.040":"37d04efc-0ab2-4499-a891-54dca1013c74","soc.062":"5e69cfac-1f68-4864-a19a-3c1bdb180100","wat.005":"1b97e47e-ca18-4e50-9aae-a2853acca3f0","cli.029":"fa6443ff-eb95-4d0f-84d2-f0c91682efdf","ene.017":"75061411-3afd-442f-b7d6-7607f97f639b"}
logging.info(long_forms)

INFO:root:Number of Carto datasets: 216
INFO:root:{'cit.022': '6b670396-c52c-430c-b5bb-20693da03b60', 'cit.025': 'd38d0d5c-31b1-47f4-9d2e-d8fba4c7d083', 'cit.028': '35ce2b98-adbb-4873-b334-d7b1cc542de7', 'cit.029': '10337db6-8321-445e-a60b-28fc1e114f29', 'cli.007': '3d2ce960-abda-4c9c-bd29-1929e9ca24c9', 'cli.008': '3a46f6b4-0eec-49d4-bbfc-e2e8f64e6117', 'com.006': '2e31a1f3-576b-46b4-84f0-3f0cc399f887', 'com.007': 'fe311144-8c0e-4440-b068-6efd057e0f6a', 'com.009': 'c61c364b-1d68-4dd9-ae3d-76c2a0022280', 'com.010': '52c55378-0484-48c3-92fc-3ee94d21c716', 'com.015': 'c18a38cd-94ff-48cd-818f-6ffb05992abb', 'com.019': '5e3a3a9f-7380-47c0-ad84-2c193861e106', 'com.028': '62c988a7-1e4d-418e-87bf-a743e24209e8', 'ene.012': 'd446a52e-c4c1-4e74-ae30-3204620a0365', 'ene.021': '2c092793-aa3a-4520-959c-ad48165dcae4', 'ene.022': 'd639909f-bcf3-4875-b8c3-35f030b68ed3', 'ene.028': 'c665f519-eef9-4f67-a8bf-7e3e6dc8bfcd', 'foo.006': '2034a766-6e8a-416d-b8ab-9b7b3e3abb15', 'foo.015': '4338471d-881a-475f-

In [161]:
cc.read('cit_029_municipal_waste')

,_2012_2015_avg,country,the_geom,yr_1990,yr_1991,yr_1992,yr_1993,yr_1994,yr_1995,yr_1996,yr_1997,yr_1998,yr_1999,yr_2000,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015
cartodb_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13,522.890000,Iceland,0106000020E6100000050000000103000000010000000F...,NaN,NaN,NaN,NaN,NaN,426.369,435.077,442.955,449.251,454.580,462.387,466.586,476.442,483.985,502.428,517.140,561.905,558.790,650.705,412.862,481.144,495.295,513.820,517.897,536.953,NaN
14,587.203000,Ireland,0106000020E6100000070000000103000000010000000E...,NaN,NaN,NaN,NaN,NaN,513.204,523.456,545.852,555.445,579.431,601.288,702.885,694.476,733.122,741.768,735.570,799.594,776.457,718.871,651.383,624.863,617.076,587.203,NaN,NaN,NaN
26,448.354667,Portugal,0106000020E61000001100000001030000000100000005...,300.504,NaN,328.561,357.563,380.323,387.386,397.757,403.602,420.760,427.058,440.355,434.316,441.009,448.691,444.988,451.779,465.496,471.146,518.256,520.074,516.134,490.433,453.257,439.688,452.119,NaN
27,314.840750,Slovak Republic,0106000020E6100000010000000103000000010000003D...,NaN,NaN,301.602,300.452,299.271,302.126,313.764,329.867,322.785,320.811,316.721,NaN,269.507,281.247,260.603,273.185,283.636,293.800,313.341,307.134,318.842,311.004,306.343,303.666,319.978,329.376
28,414.217000,Slovenia,0106000020E61000000100000001030000000100000007...,NaN,NaN,NaN,NaN,NaN,596.693,590.106,588.872,584.479,548.964,512.493,478.245,406.901,417.684,485.332,494.090,516.038,524.951,541.256,523.592,489.884,415.163,361.826,414.445,432.528,448.069
29,451.394500,Spain,0106000020E61000001700000001030000000100000028...,NaN,NaN,NaN,NaN,NaN,471.608,494.198,516.587,520.256,565.562,609.805,609.178,637.416,646.266,600.712,588.215,590.822,578.168,550.570,541.499,510.592,485.114,468.207,454.663,448.441,434.267
30,446.197000,Sweden,0106000020E61000002900000001030000000100000015...,373.883,NaN,NaN,NaN,399.397,385.751,391.133,415.778,437.127,428.320,427.520,439.211,464.876,463.887,459.714,476.850,489.696,485.617,483.164,469.507,438.797,449.375,450.144,450.163,437.851,446.630
31,724.606500,Switzerland,0106000020E610000001000000010300000001000000EE...,610.970,607.531,595.852,596.720,595.027,602.214,605.216,605.792,614.543,637.627,658.524,663.378,677.580,669.846,663.092,664.236,712.192,723.734,739.179,705.206,710.912,692.331,700.924,711.903,743.784,741.815
32,406.146000,Turkey,0106000020E61000000600000001030000000100000011...,NaN,398.091,NaN,NaN,398.525,455.756,483.723,518.719,527.870,480.873,476.390,476.180,469.668,465.328,439.893,458.124,434.111,432.817,400.469,419.161,406.510,415.797,409.520,406.550,406.097,402.417


In [56]:
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

In [128]:
# Load config options
try:
    table_info = json.load(open('./table_info.json', 'r'))
except:
    table_info = {}
table_info

{'%20soc_023_fragile_states_index': {'error': '* Problem fetching %20soc_023_fragile_states_index from Carto server *'},
 'cit_022_road_traffic_deaths_by_country': {'config_options': {'_incl': True,
   'add_ind': True,
   'country_code': False,
   'country_name': 'country',
   'id_cols': ['country'],
   'prefixes': ['estimated_number_of_road_traffic_deaths']}},
 'cit_025_urban_population': {'config_options': {'_incl': True,
   'add_ind': False,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'prefixes': ['yr']}},
 'cit_028_percent_urban_slums_edit': {'config_options': {'_incl': True,
   'add_ind': False,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'prefixes': ['yr']}},
 'cit_029_municipal_waste': {'config_options': {'_incl': True,
   'add_ind':

In [23]:
tables = {}
tables['geometry'] = {
    'data':cc.read('wri_countries_a')
}
table_info['geometry'] = {
    'config_options':{}
}
tables['country_aliases'] = {
    'data':cc.read('rw_aliasing_countries')
}
table_info['country_aliases'] = {
    'config_options':{}
}

for rw_id, api_id in long_forms.items():
    logging.info(rw_id)

    # Fetch table name
    try:
        table_name = carto_data.loc[api_id]['table_name']
    except:
        msg = '***** ' + rw_id + ' not in carto_data *****'
        table_info[table_name] = {
            'error':msg
        }
        logging.info(msg)


    if table_name in table_info:
        if 'config_options' in table_info[table_name]:
            if 'prefixes' in table_info[table_name]['config_options']:
                if table_info[table_name]['config_options']['prefixes'] in [[u'--'], [u'already long']]:
                    logging.info('not fetching data for ' + table_name)
                    continue

    # Fetch data
    try:
        tables[table_name] = {
            'data':cc.read(table_name)
        }
        if 'config_options' not in table_info[table_name]:
            logging.info('updating table_info for ' + table_name)
            table_info[table_name] = {
                'config_options':{}
            }
    except:
        msg = '***** Problem fetching ' + table_name + ' from Carto server *****'
        if table_name in table_info:
            if 'error' in table_info[table_name]:
                logging.info(table_info[table_name]['error'])
        else:
            table_info[table_name] = {
                'error':msg
            }
            logging.info(msg)


for name, info in tables.items():
    logging.info('table name: ' + name)
    try:
        logging.info('table shape: ' + str(info['data'].shape))
    except:
        logging.info(info['error'])

INFO:root:cit.022
INFO:root:cit.025
INFO:root:cit.028
INFO:root:cit.029
INFO:root:cli.007
INFO:root:not fetching data for cli_007_greenhouse_gas_emissions_projections
INFO:root:cli.008
INFO:root:not fetching data for cli_008_greenhouse_gas_emissions_country_sector
INFO:root:com.006
INFO:root:com.007
INFO:root:not fetching data for com_007_fdi_index
INFO:root:com.009
INFO:root:not fetching data for com_009_1_material_flows_main
INFO:root:com.010
INFO:root:com.015
INFO:root:com.019
INFO:root:not fetching data for com_019_ecological_footprint_edit
INFO:root:com.028
INFO:root:not fetching data for com_028_effect_of_ag_prices_on_commodity_prices
INFO:root:ene.012
INFO:root:ene.021
INFO:root:ene.022
INFO:root:not fetching data for ene_022_se4all_tracking_goals
INFO:root:ene.028
INFO:root:foo.006
INFO:root:not fetching data for foo_006_effects_climate_change_food_production
INFO:root:foo.015
INFO:root:foo.019
INFO:root:not fetching data for foo_019_child_malnutrition_edit
INFO:root:foo.040
IN

In [32]:
# Add config options
for name, info in tables.items():
    logging.info(name)
    logging.info(info['data'].columns)
    print

    ## To assign prefixes, uncomment
    '''
    prefixes = input('time column prefixes?')
    table_info[name]['config_options']['prefixes'] = [pfx.strip() for pfx in prefixes.split(',')]
    '''

    ## To determine whether column prefixes should be added as a new column, uncomment
    '''
    add_indicator_column = input('Add prefix as indicator? type anything for True, leave blank for False')
    if add_indicator_column != '':
        table_info[name]['config_options']['add_ind'] = True
    else:
        table_info[name]['config_options']['add_ind'] = False
    '''

    ## To determine whether the column should have a _ after the prefix, uncomment
    '''
    _included = input('Is there a _ after the prefix? type anything for True, leave blank for False')
    if _included != '':
        table_info[name]['config_options']['_incl'] = True
    else:
        table_info[name]['config_options']['_incl'] = False
    '''

    ## To determine identifying info column (likely country code), uncomment
    '''
    country_code = input('country code column? leave blank if needs country code added')
    if country_code != '':
        table_info[name]['config_options']['country_code'] = country_code
    else:
        table_info[name]['config_options']['country_code'] = False

    country_name = input('country name column? leave blank if needs country name added')
    if country_name != '':
        table_info[name]['config_options']['country_name'] = country_name
    else:
        table_info[name]['config_options']['country_name'] = False
    '''
    
    ## To determine id columns to use, uncomment
    '''
    id_cols = input('id columns? separate by comma with a space')
    table_info[name]['config_options']['id_cols'] = [col.strip() for col in id_cols.split(',')]
    '''

INFO:root:geometry
INFO:root:Index(['_primary', 'abbrev', 'adm0_a3', 'continent', 'formal_nam', 'iso_a2',
       'iso_a3', 'iso_n3', 'iso_short', 'name', 'name_sort', 'postal',
       'short_name', 'sov_a3', 'sovereign', 'the_geom', 'type', 'un_ar',
       'un_en', 'un_es', 'un_fr', 'un_n3', 'un_region', 'un_ru', 'un_subregi',
       'un_zh', 'wb_a2', 'wb_a3', 'wb_region'],
      dtype='object')


id columns? separate by comma with a space


INFO:root:country_aliases
INFO:root:Index(['alias', 'iso', 'name', 'the_geom'], dtype='object')


id columns? separate by comma with a space


INFO:root:cit_022_road_traffic_deaths_by_country
INFO:root:Index(['cartodb_georef_status', 'country', 'death_rate_per_100000',
       'estimated_number_of_road_traffic_deaths_2013', 'the_geom'],
      dtype='object')


...

id columns? separate by comma with a spacecountry_code, country_name


INFO:root:wat_005_improved_water_access
INFO:root:Index(['cartodb_georef_status', 'country_code', 'country_name', 'rural_1990',
       'rural_1991', 'rural_1992', 'rural_1993', 'rural_1994', 'rural_1995',
       'rural_1996', 'rural_1997', 'rural_1998', 'rural_1999', 'rural_2000',
       'rural_2001', 'rural_2002', 'rural_2003', 'rural_2004', 'rural_2005',
       'rural_2006', 'rural_2007', 'rural_2008', 'rural_2009', 'rural_2010',
       'rural_2011', 'rural_2012', 'rural_2013', 'rural_2014', 'rural_2015',
       'the_geom', 'total_1990', 'total_1991', 'total_1992', 'total_1993',
       'total_1994', 'total_1995', 'total_1996', 'total_1997', 'total_1998',
       'total_1999', 'total_2000', 'total_2001', 'total_2002', 'total_2003',
       'total_2004', 'total_2005', 'total_2006', 'total_2007', 'total_2008',
       'total_2009', 'total_2010', 'total_2011', 'total_2012', 'total_2013',
       'total_2014', 'total_2015', 'urban_1990', 'urban_1991', 'urban_1992',
       'urban_1993', 'urban

id columns? separate by comma with a spacecountry_code, country_name


INFO:root:cli_029_vulnerability_to_cc
INFO:root:Index(['cartodb_georef_status', 'country', 'gain_1995', 'gain_1996',
       'gain_1997', 'gain_1998', 'gain_1999', 'gain_2000', 'gain_2001',
       'gain_2002', 'gain_2003', 'gain_2004', 'gain_2005', 'gain_2006',
       'gain_2007', 'gain_2008', 'gain_2009', 'gain_2010', 'gain_2011',
       'gain_2012', 'gain_2013', 'gain_2014', 'gain_2015', 'iso4',
       'readiness_1995', 'readiness_1996', 'readiness_1997', 'readiness_1998',
       'readiness_1999', 'readiness_2000', 'readiness_2001', 'readiness_2002',
       'readiness_2003', 'readiness_2004', 'readiness_2005', 'readiness_2006',
       'readiness_2007', 'readiness_2008', 'readiness_2009', 'readiness_2010',
       'readiness_2011', 'readiness_2012', 'readiness_2013', 'readiness_2014',
       'readiness_2015', 'the_geom', 'vulnerability_1995',
       'vulnerability_1996', 'vulnerability_1997', 'vulnerability_1998',
       'vulnerability_1999', 'vulnerability_2000', 'vulnerability_2001',


id columns? separate by comma with a spacecountry


In [159]:
json.dump(table_info, open('./table_info.json', 'w'))
table_info = json.load(open('./table_info.json', 'r'))
table_info

{'%20soc_023_fragile_states_index': {'error': '* Problem fetching %20soc_023_fragile_states_index from Carto server *'},
 'cit_022_road_traffic_deaths_by_country': {'config_options': {'_incl': True,
   'add_ind': True,
   'country_code': False,
   'country_name': 'country',
   'id_cols': ['country'],
   'prefixes': ['estimated_number_of_road_traffic_deaths']}},
 'cit_025_urban_population': {'config_options': {'_incl': True,
   'add_ind': False,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'prefixes': ['yr']}},
 'cit_028_percent_urban_slums_edit': {'config_options': {'_incl': True,
   'add_ind': False,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'prefixes': ['yr']}},
 'cit_029_municipal_waste': {'config_options': {'_incl': True,
   'add_ind':

In [68]:
# Use known prefixes to reformat tables
'''
ignore_columns = ['the_geom', 'cartodb_georef_status', ]
def pick_id_cols(col, prefixes):
    if col in ignore_columns:
        return False
    for pfx in prefixes:
        if pfx+'_' in col:
            return False
    return True
'''

def pick_value_cols(col, prefixes, use_, only=None):
    if use_:
        seen = [True if pfx+'_' in col else False for pfx in prefixes]
    else:
        seen = [True if pfx in col else False for pfx in prefixes]

    if only:
        if sum(seen) == 1:
            # Only one match, keep if it matches 'only'
            if seen[prefixes.index(only)]:
                return True
        elif sum(seen) > 1:
            # More than one match, keep longest if it matches 'only'
            seen_pfxs = [_[0] for _ in list(zip(prefixes,seen)) if _[1]]
            seen_lengths = [len(_) for _ in seen_pfxs]
            max_index = seen_lengths.index(max(seen_lengths))
            if prefixes.index(seen_pfxs[max_index]) == prefixes.index(only):
                return True
    else:
        if sum(seen):
            return True
    return False

def prepare_date(date, pfx, use_):
    if use_:
        return date[date.index(pfx) + len(pfx) + 1:]
    else:
        return date[date.index(pfx) + len(pfx):]

In [155]:
for name, info in tables.items():
    if not table_info[name]['config_options'].get('prefixes', False):
        continue
    if table_info[name]['config_options']['prefixes'][0] in ['--', 'already long']:
        logging.info('skipping ' + name)
        continue

    logging.info('processing ' + name)
    prefixes = table_info[name]['config_options'].get('prefixes', [])
    id_cols = table_info[name]['config_options'].get('id_cols', [])
    use_ = table_info[name]['config_options'].get('_incl', False)
    add_ind = table_info[name]['config_options'].get('add_ind', False)

    logging.info('options:')
    logging.info(table_info[name]['config_options'])
    logging.info('initial shape: ' + str(info['data'].shape))
    logging.info('')
    
    # If adding info from pfx to indicator, melt one set at a time and join on 'variable' column
    if add_ind:
        df = pd.DataFrame(columns = id_cols + ['variable'])
        for pfx in prefixes:
            value_cols = [col for col in info['data'].columns if pick_value_cols(col, prefixes, use_, only=pfx)]
            _df = pd.melt(info['data'], id_vars=id_cols, value_vars=value_cols)
            _df['variable'] = [prepare_date(date, pfx, use_) for date in _df['variable']]
            col_names = list(_df.columns)
            col_names[col_names.index('value')] = pfx + '_data'
            _df.columns = col_names
            df = df.merge(_df, on=id_cols  + ['variable'], how='outer')
        logging.info('final shape of ' + name + ': ' + str(df.shape))
        logging.info('final columns: ' + str(df.columns))

    # If not preserving info from pfx, melt all at once
    else:
        pfx = prefixes[0]
        value_cols = [col for col in info['data'].columns if pick_value_cols(col, prefixes,use_)]
        df = pd.melt(info['data'], id_vars=id_cols, value_vars=value_cols)
        col_names = list(df.columns)
        col_names[col_names.index('value')] = pfx + '_data'
        df.columns = col_names
        df['variable'] = [prepare_date(date, pfx, use_) for date in df['variable']]
        logging.info('final shape of ' + name + ': ' + str(df.shape))

    new_cols = ['datetime' if col=='variable' else col for col in df.columns]
    df.columns = new_cols
    tables[name]['long_data'] = df

INFO:root:processing cit_022_road_traffic_deaths_by_country
INFO:root:options:
INFO:root:{'_incl': True, 'add_ind': True, 'prefixes': ['estimated_number_of_road_traffic_deaths'], 'country_code': False, 'country_name': 'country', 'id_cols': ['country']}
INFO:root:initial shape: (179, 5)
INFO:root:
INFO:root:final shape of cit_022_road_traffic_deaths_by_country: (179, 3)
INFO:root:final columns: Index(['country', 'variable', 'estimated_number_of_road_traffic_deaths_data'], dtype='object')
INFO:root:processing cit_025_urban_population
INFO:root:options:
INFO:root:{'_incl': True, 'add_ind': False, 'prefixes': ['yr'], 'country_code': 'country_code', 'country_name': 'country_name', 'id_cols': ['country_code', 'country_name', 'indicator_code', 'indicator_name']}
INFO:root:initial shape: (218, 63)
INFO:root:
INFO:root:final shape of cit_025_urban_population: (12426, 6)
INFO:root:processing cit_028_percent_urban_slums_edit
INFO:root:options:
INFO:root:{'_incl': True, 'add_ind': False, 'prefixes

Explore geometries and aliasing table

In [208]:
tables['geometry']['data'].head(1)

,_primary,abbrev,adm0_a3,continent,formal_nam,iso_a2,iso_a3,iso_n3,iso_short,name,name_sort,postal,short_name,sov_a3,sovereign,the_geom,type,un_ar,un_en,un_es,un_fr,un_n3,un_region,un_ru,un_subregi,un_zh,wb_a2,wb_a3,wb_region
cartodb_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
52,1,Egypt,EGY,Africa,Arab Republic of Egypt,EG,EGY,818,Egypt,Egypt,"Egypt, Arab Rep.",EG,Egypt,EGY,Egypt,0106000020E6100000080000000103000000010000000A...,Sovereign country,رصم,Egypt,Egipto,Égypte (l') [fém.],818,Africa,Египет,Northern Africa,埃及,EG,EGY,Middle East & North Africa


In [210]:
tables['country_aliases']['data'].head(1)

,alias,iso,name,the_geom
cartodb_id,,,,
1,Korea North,PRK,North Korea,None


In [279]:
tables['country_aliases']['data'][tables['country_aliases']['data']['alias']=='Madagascar']
#tables['geometry']['data'][tables['geometry']['data']['adm0_a3']=='MLI']

,alias,iso,name,the_geom
110,Madagascar,MDG,Madagascar,NaN


In [278]:
new_aliases = tables['geometry']['data'][['adm0_a3', 'name']].copy()
new_aliases['alias'] = new_aliases['name']
cols = str(list(new_aliases.columns)).replace('adm0_a3', 'iso')[1:-1].replace('\'','').split(',')
cols = [col.strip() for col in cols]
new_aliases.columns = cols
with_country_geometry_info = tables['country_aliases']['data'].append(new_aliases)
with_country_geometry_info[with_country_geometry_info['iso']=='PLW']
sum(with_country_geometry_info.duplicated(subset=['alias']))

tables['country_aliases']['data'] = with_country_geometry_info

In [289]:
# Adding geometries to long data
def join_data_with_aliases(data, aliases, left_on='country', right_on='name'):
    data = data.copy()
    data[left_on] = data[left_on].apply(lambda item: item.strip())
    data_with_aliases = data.merge(aliases, 
                                     left_on = left_on,
                                     right_on = right_on,
                                     how='left')
    
    logging.info('merged data:')
    logging.info(data_with_aliases.head(5))
    null_aliases = pd.isnull(data_with_aliases[right_on])
    
    logging.info('number null aliases')
    logging.info(sum(null_aliases))
    logging.info('--')
    
    return data_with_aliases, null_aliases

'''
def transfer_columns_between_tables(src_df, dst_df, columns_to_xfer=None):
    """
    Inputs:
    * src_df and dst_df with same index
    * list of columns from src_df to transfer to dst_df
    
    Outputs:
    * dst_df with new columns from the src_df
    """
    
    dst_df = dst_df.copy()
    try:
        info = src_df.loc[dst_df.index, columns_to_xfer]
        dst_df[columns_to_xfer] = info
        return(dst_df)
    except:
        print("ERROR")
        print("Possible source: There is an element in the metadata sheet that is not in the tracking sheet")
        print("Possible source: One of more of the given columns to transfer is not in the source df")
        return(None)
'''

def match_iso_to_name(names, aliases, match_col='alias'):
    logging.info('matching parameters')
    logging.info(names.head(5))
    logging.info(aliases.head(5))
    logging.info(match_col)
    logging.info('--')
    logging.info('--')

    alias_indices = []
    poss_aliases = list(aliases[match_col])
    for name in names:
        try:
            alias_indices.append(poss_aliases.index(name))
        except:
            alias_indices.append('***')
 
    isos = list(aliases['iso'])
    
    matching_aliases = []
    for index in alias_indices:
        try:
            matching_aliases.append(isos[index])
        except:
            matching_aliases.append('***')
            
    return matching_aliases

In [290]:
name = 'cit_022_road_traffic_deaths_by_country'
c_name = 'country'
aliases = tables['country_aliases']['data']

df = pd.DataFrame(tables[name]['long_data'][c_name])

a, b = join_data_with_aliases(df, aliases, left_on = c_name, right_on = 'alias' )
print(a[b])

INFO:root:merged data:
INFO:root:      country       alias  iso        name the_geom
0        Mali        Mali  MLI        Mali     None
1        Mali        Mali  MLI        Mali      NaN
2  Madagascar  Madagascar  MDG  Madagascar      NaN
3    Malaysia    Malaysia  MYS    Malaysia      NaN
4     Liberia     Liberia  LBR     Liberia      NaN
INFO:root:number null aliases
INFO:root:1
INFO:root:--


          country alias  iso name the_geom
186  Cook Islands   NaN  NaN  NaN      NaN


In [291]:
# Tracking all mis-matched names
missed_names = []

for name, info in tables.items():
    if not table_info[name]['config_options'].get('prefixes', False):
        logging.info('skipping ' + name + ', no prefixes available. Probably already in long form.')
        continue
    if table_info[name]['config_options']['prefixes'][0] in ['--', 'already long']:
        logging.info('skipping ' + name + ', needs a second look, or is already long')
        continue
    
    logging.info('Processing table ' + name)
    logging.info('Table head:')
    logging.info(info['long_data'].head(5))
    c_code = table_info[name]['config_options']['country_code']
    c_name = table_info[name]['config_options']['country_name']
    logging.info('c_code: ***' + str(c_code) + '***')
    logging.info('c_name: ***' + c_name + '***')
    logging.info('--')
    logging.info('--')
    
    # If country code is supplied, check if all country codes in long_data 
    # have a matching iso in country_aliases
    
    aliases = tables['country_aliases']['data']
    data = info['long_data']
    
    # If country name is supplied, check how many match up with alias/name in country_aliases
    if c_name:     
        data_c_name = pd.DataFrame(data[c_name],columns=[c_name])
        data_with_aliases, null_aliases = join_data_with_aliases(data_c_name,aliases, left_on=c_name, right_on='alias')
        if sum(null_aliases):
            no_alias_match = data_with_aliases.loc[null_aliases]
            logging.info('missed aliases, matching on column "alias" of country_aliases')
            logging.info(no_alias_match[c_name].unique())
            missed_names.extend(no_alias_match[c_name].unique())
            
        '''
        data_with_aliases, null_aliases = join_data_with_aliases(data_c_name,aliases, left_on=c_name, right_on='name')
        if sum(null_aliases):
            no_alias_match = data_with_aliases.loc[null_aliases]
            logging.info('missed aliases, matching on column "name" of country_aliases')            
            logging.info(no_alias_match[c_name].unique())
        '''

    if c_code:
        logging.info('already has an iso3 code, in column ' + c_code)
        isos_from_data = pd.DataFrame(data[c_code])
        logging.info(isos_from_data.head(5))
        
        isos_from_data = isos_from_data.set_index(c_code)
        
        aliases_by_isos = isos_from_data.merge(aliases,
                   left_index=True,
                   right_on='iso', 
                   how='left')
        
        null_isos = pd.isnull(aliases_by_isos['iso'])
        no_alias_iso_match = aliases_by_isos[null_isos.values]
        logging.info('no match for these isos in the data being processed: ' + str(no_alias_iso_match))
    else:
        ### Use the names now checked for to assign a c_code column
        ### THIS IS THE ONLY PLACE DATA IS ALTERED
        # If there was no c_code, there must be a c_name
        data['new_country_code'] = match_iso_to_name(data[c_name], aliases, match_col='alias')
        
    ### BEFORE HERE, ensure data has the updated geometry column attached
    tables[name]['long_data'] = data


INFO:root:skipping geometry, no prefixes available. Probably already in long form.
INFO:root:skipping country_aliases, no prefixes available. Probably already in long form.
INFO:root:Processing table cit_022_road_traffic_deaths_by_country
INFO:root:Table head:
INFO:root:      country datetime estimated_number_of_road_traffic_deaths_data  \
0        Mali     2013                        3 920 [3 193 - 4 648]   
1  Madagascar     2013                        6 506 [5 361 - 7 650]   
2    Malaysia     2013                        7 129 [6 050 - 8 209]   
3     Liberia     2013                        1 448 [1 144 - 1 752]   
4    Maldives     2013                                           12   

  new_country_code  
0              MLI  
1              MDG  
2              MYS  
3              LBR  
4              MDV  
INFO:root:c_code: ***False***
INFO:root:c_name: ***country***
INFO:root:--
INFO:root:--
INFO:root:merged data:
INFO:root:      country       alias  iso        name the_geom
0  

In [292]:
missed_names = list(set(missed_names))
print(len(missed_names))
print(missed_names)

139
['Middle East & North Africa (IDA & IBRD countries)', 'Other small states', 'European Union', "Democratic People's Republic of Korea", 'Upper middle income', 'IDA only', 'Pitcairn', 'Jersey', 'Saint Pierre et Miquelon', 'Aruba', 'Bolivia (Plurinational State of)', 'Venezuela, Bolivarian Republic of', 'Kosovo', 'Puerto Rico', 'Virgin Islands', 'United States Virgin Islands', 'Channel Islands', 'Åland Islands', 'St. Martin (French part)', 'West Bank and Gaza', 'Latin America & Caribbean (excluding high income)', 'Europe & Central Asia (IDA & IBRD countries)', 'East Asia & Pacific (IDA & IBRD countries)', 'Libyan Arab Jamahiriya', 'Congo (Democratic Republic of the)', 'Latin America & Caribbean', 'Heavily indebted poor countries (HIPC)', 'IDA & IBRD total', 'Isle of Man', 'Hong Kong', 'French Guiana', 'Hong Kong SAR, China', 'Low & middle income', 'IBRD only', 'Cook Islands', 'Lao PDR', 'Korea, Dem. People\x92s Rep.', 'Sint Maarten (Dutch part)', 'Saint-Martin (French part)', 'Montser

In [295]:
# Code to add entries for capitalized country names which are otherwise in the alias table
_aliases = [alias.lower() for alias in aliases['alias']]
additional_aliases = []
for name in missed_names:
    _name = name.lower()
    if _name in _aliases:
        new_alias = aliases.iloc[_aliases.index(_name)]
        new_alias['alias'] = name
        additional_aliases.append(new_alias)

add_aliases = pd.DataFrame(additional_aliases)
tables['country_aliases']['data'] = tables['country_aliases']['data'].append(add_aliases)

In [296]:
tables['country_aliases']['data'].shape

(636, 4)

In [299]:
for name, info in tables.items():
    logging.info(name)
    for key, data in info.items():
        if key == 'long_data':
            logging.info(data.head())
            write_to_S3(data,s3_bucket,s3_folder+name+'_long')
            logging.info('saved ' + name + ' long data to s3')
            cc.write(data, name + '_longform', overwrite=True)
            logging.info('saved ' + name + ' long data to Carto')
            #logging.info('failed to write table ' + name)

INFO:root:geometry
INFO:root:country_aliases
INFO:root:cit_022_road_traffic_deaths_by_country
INFO:root:      country datetime estimated_number_of_road_traffic_deaths_data  \
0        Mali     2013                        3 920 [3 193 - 4 648]   
1  Madagascar     2013                        6 506 [5 361 - 7 650]   
2    Malaysia     2013                        7 129 [6 050 - 8 209]   
3     Liberia     2013                        1 448 [1 144 - 1 752]   
4    Maldives     2013                                           12   

  new_country_code  
0              MLI  
1              MDG  
2              MYS  
3              LBR  
4              MDV  
INFO:root:saved cit_022_road_traffic_deaths_by_country long data to s3
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/cartoframes/context.py:511: UserWarning: DataFrame written to CARTO but the table schema failed to update to match DataFrame. All columns in CARTO table have data type `text`. CARTO error: `['syntax err

Table successfully written to CARTO: https://wri-rw.carto.com/dataset/cit_022_road_traffic_deaths_by_country_longform


INFO:root:saved cit_025_urban_population long data to s3
INFO:root:saved cit_025_urban_population long data to Carto
INFO:root:cit_028_percent_urban_slums_edit
INFO:root:  country_code    country_name     indicator_code  \
0          AFG     Afghanistan  EN.POP.SLUM.UR.ZS   
1          INX  Not classified  EN.POP.SLUM.UR.ZS   
2          IND           India  EN.POP.SLUM.UR.ZS   
3          IRL         Ireland  EN.POP.SLUM.UR.ZS   
4          ITA           Italy  EN.POP.SLUM.UR.ZS   

                                      indicator_name datetime yr_data  
0  Population living in slums (% of urban populat...     1960          
1  Population living in slums (% of urban populat...     1960          
2  Population living in slums (% of urban populat...     1960          
3  Population living in slums (% of urban populat...     1960          
4  Population living in slums (% of urban populat...     1960          


Table successfully written to CARTO: https://wri-rw.carto.com/dataset/cit_025_urban_population_longform


INFO:root:saved cit_028_percent_urban_slums_edit long data to s3
INFO:root:saved cit_028_percent_urban_slums_edit long data to Carto
INFO:root:cit_029_municipal_waste
INFO:root:   _2012_2015_avg          country datetime  yr_data new_country_code
0      522.890000          Iceland     1990      NaN              ISL
1      587.203000          Ireland     1990      NaN              IRL
2      448.354667         Portugal     1990  300.504              PRT
3      314.840750  Slovak Republic     1990      NaN              SVK
4      414.217000         Slovenia     1990      NaN              SVN
INFO:root:saved cit_029_municipal_waste long data to s3


...

Table successfully written to CARTO: https://wri-rw.carto.com/dataset/soc_040_improved_sanitation_longform


INFO:root:saved wat_005_improved_water_access long data to s3
INFO:root:saved wat_005_improved_water_access long data to Carto
INFO:root:cli_029_vulnerability_to_cc
INFO:root:         country datetime  gain_data  readiness_data  vulnerability_data  \
0        Andorra     1995        NaN             NaN                 NaN   
1  Liechtenstein     1995        NaN             NaN                 NaN   
2         Monaco     1995        NaN             NaN                 NaN   
3          Nauru     1995        NaN             NaN                 NaN   
4     San Marino     1995        NaN             NaN                 NaN   

  new_country_code  
0              AND  
1              LIE  
2              MCO  
3              NRU  
4              SMR  


Table successfully written to CARTO: https://wri-rw.carto.com/dataset/wat_005_improved_water_access_longform


INFO:root:saved cli_029_vulnerability_to_cc long data to s3
INFO:root:saved cli_029_vulnerability_to_cc long data to Carto


Table successfully written to CARTO: https://wri-rw.carto.com/dataset/cli_029_vulnerability_to_cc_longform
